In [6]:
import pandas as pd 
import numpy as np
import Levenshtein as lv
import re, math
from collections import Counter
import os
import sys

In [27]:
path = "c:/Users/IBSS/Documents/arastirma/dynamic_text_similarity"
path = path + "/data/Muhtp_Spart.xlsx"
dataset = pd.read_excel(path) 

In [28]:
dataset["Col"] = dataset["MANDT"].map(str) + dataset["NAME1"].map(str) 

In [29]:
 #df = df[[self.unique_column, self.column, grup_lev, score_lev, grup_cos, score_cos, grup_jac, score_jac]] 
dataset = dataset[["Col", "KAYIT SIRASI"]]

In [30]:
dataset

,Col,KAYIT SIRASI
0,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,1
1,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,2
2,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,3
3,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,4
4,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,5
...,...,...
73261,1103018 - Diyarbakır Dağıtım Merkezi,73262
73262,1103018 - Diyarbakır Dağıtım Merkezi,73263
73263,1103018 - Diyarbakır Dağıtım Merkezi,73264
73264,1103019-Selçuk Dağıtım Merkezi,73265


In [31]:
dataset = pd.DataFrame(dataset["Col"], index = dataset['KAYIT SIRASI'])

In [32]:
dataset

,Col
KAYIT SIRASI,
1,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
2,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
3,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
4,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
5,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
...,...
73262,1103018 - Diyarbakır Dağıtım Merkezi
73263,1103018 - Diyarbakır Dağıtım Merkezi
73264,1103019-Selçuk Dağıtım Merkezi


In [16]:
dataset

,ESLESME GRUBU,BENZERLIK GRUBU,BENZERLIK YUZDESI,KAYIT SIRASI,BENZEDIGI_KAYIT_NUMARASI,MANDT,KUNNR,VKORG,VTWEG,SPART,PARVW,NAME1,NAME2,PARZA,SOTNM,DKTNM,MRTNM,BLTNM,Col
0,NaN,NaN,NaN,1,NaN,110,1000000,1000,10,10,AG,A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,NaN,0,Eti Gıda SO,Yurtiçi,Sipariş Veren,Faaliyet İçi Sat.,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
1,NaN,NaN,NaN,2,NaN,110,1000000,1000,10,10,RE,A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,NaN,0,Eti Gıda SO,Yurtiçi,Fatura alıcısı,Faaliyet İçi Sat.,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
2,NaN,NaN,NaN,3,NaN,110,1000000,1000,10,10,RG,A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,NaN,0,Eti Gıda SO,Yurtiçi,Ödeyen,Faaliyet İçi Sat.,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
3,NaN,NaN,NaN,4,NaN,110,1000000,1000,10,10,WE,A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,NaN,0,Eti Gıda SO,Yurtiçi,Malı Teslim Alan,Faaliyet İçi Sat.,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
4,NaN,NaN,NaN,5,NaN,110,1000000,1000,10,20,AG,A.S.WATSON GÜZEL.BAKIM ÜR.A.S.,NaN,0,Eti Gıda SO,Yurtiçi,Sipariş Veren,Faaliyet Dışı Sat.,110A.S.WATSON GÜZEL.BAKIM ÜR.A.S.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73261,NaN,NaN,NaN,73262,NaN,110,SNM3018,1000,30,10,RE,3018 - Diyarbakır Dağıtım Merkezi,NaN,0,Eti Gıda SO,Stok Nakli,Fatura alıcısı,Faaliyet İçi Sat.,1103018 - Diyarbakır Dağıtım Merkezi
73262,NaN,NaN,NaN,73263,NaN,110,SNM3018,1000,30,10,RG,3018 - Diyarbakır Dağıtım Merkezi,NaN,0,Eti Gıda SO,Stok Nakli,Ödeyen,Faaliyet İçi Sat.,1103018 - Diyarbakır Dağıtım Merkezi
73263,NaN,NaN,NaN,73264,NaN,110,SNM3018,1000,30,10,WE,3018 - Diyarbakır Dağıtım Merkezi,NaN,0,Eti Gıda SO,Stok Nakli,Malı Teslim Alan,Faaliyet İçi Sat.,1103018 - Diyarbakır Dağıtım Merkezi
73264,NaN,NaN,NaN,73265,NaN,110,SNM3019,1000,30,10,AG,3019-Selçuk Dağıtım Merkezi,NaN,0,Eti Gıda SO,Stok Nakli,Sipariş Veren,Faaliyet İçi Sat.,1103019-Selçuk Dağıtım Merkezi


In [12]:
dataset1 = dataset.MANDT

In [14]:
dataset1 = dataset1.merge(dataset.NAME1, how = 'outer', left_index = True, right_index = True)

AttributeError: 'Series' object has no attribute 'merge'

In [ ]:
NAME1

In [13]:
dataset1

0        110
1        110
2        110
3        110
4        110
        ... 
73261    110
73262    110
73263    110
73264    110
73265    110
Name: MANDT, Length: 73266, dtype: int64

In [ ]:
df_merged = df_merged.merge(sum_sim[k], how = 'outer', left_index = True, right_index = True)